In [4]:
import pandas as pd
import numpy as np

# Load the CSV file using the relative path from the scoring folder
df = pd.read_csv("../CleanedData/accountlevelfinal.csv")
df.head()


,Season,AccountNumber,SingleGameTickets,PartialPlanTickets,GroupTickets,AvgSpend,GamesAttended,FanSegment,DistanceToArena,BasketballPropensity,...,Tier_AB_Fraction,MostAttendedTier,WeekendFraction,TierA_Weekday_Fraction,TierB_Weekday_Fraction,TierC_Weekday_Fraction,TierD_Weekday_Fraction,TierAB_Weekday_Fraction,TierCD_Weekday_Fraction,GiveawayFraction
0,2023,1,0,0,0,467.0,0,F,12.0,872.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,1.0
1,2023,2,2,0,0,116.0,1,A,47.0,485.0,...,1.0,Tier_A,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0
2,2023,3,3,0,0,107.0,1,B,6.0,896.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0
3,2023,4,0,0,3,27.0,1,C,3.0,467.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0
4,2023,5,0,0,2,14.0,1,A,4.0,582.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0


In [5]:
# For the fan segment bonus, assign 0.5 if the FanSegment is 'D'; otherwise assign 0.
df['FanSegmentBonus'] = df['FanSegment'].apply(lambda x: 0.5 if x == 'D' else 0)


In [6]:
# Calculate the ValuePlanScore based on the given formula:
# ValuePlanScore = 0.25*(1 – AvgSpend_norm)
#                + 0.25*(TierCD_Weekday_Fraction)
#                + 0.15*(DistanceToArena_norm)
#                + 0.15*(1 – GamesAttended_norm)
#                + 0.20*(FanSegmentBonus)
df['ValuePlanScore'] = (
    0.25 * (1 - df['AvgSpend_norm']) +
    0.25 * (df['TierCD_Weekday_Fraction']) +
    0.15 * (df['DistanceToArena_norm']) +
    0.15 * (1 - df['GamesAttended_norm']) +
    0.20 * (df['FanSegmentBonus'])
)

# Display the new column along with its components for verification
df[['AvgSpend_norm', 'TierCD_Weekday_Fraction', 'DistanceToArena_norm', 'GamesAttended_norm', 'FanSegmentBonus', 'ValuePlanScore']].head()


,AvgSpend_norm,TierCD_Weekday_Fraction,DistanceToArena_norm,GamesAttended_norm,FanSegmentBonus,ValuePlanScore
0,1.000000,0.5,0.042254,0.0,0.0,0.281338
1,0.590331,0.0,0.165493,0.2,0.0,0.247241
2,0.544529,0.5,0.021127,0.2,0.0,0.362037
3,0.137405,0.5,0.010563,0.2,0.0,0.462233
4,0.071247,0.5,0.014085,0.2,0.0,0.479301


In [7]:
# Create a mapping for FanSegment bonus values for the Marquee plan.
bonus_map = {'G': 1.0, 'F': 0.75, 'C': 0.4, 'E': 0.2}

# Map the FanSegment column to bonus values.
# If the FanSegment is not one of these, we assign a default bonus of 0.
df['FanSegmentBonus_Marquee'] = df['FanSegment'].map(bonus_map).fillna(0)


In [9]:
# Calculate MarqueePlanScore using the formula:
# MarqueePlanScore = 0.30*(AvgSpend_norm)
#                   + 0.30*(TierAB_Fraction)
#                   + 0.20*(BasketballPropensity_norm)
#                   + 0.20*(FanSegmentBonus_Marquee)

df['MarqueePlanScore'] = (
    0.30 * df['AvgSpend_norm'] +
    0.30 * df['Tier_AB_Fraction'] +
    0.20 * df['BasketballPropensity_norm'] +
    0.20 * df['FanSegmentBonus_Marquee']
)


In [11]:
# Display the key columns to verify the new MarqueePlanScore calculation
df[['AvgSpend_norm', 'Tier_AB_Fraction', 'BasketballPropensity_norm', 
    'FanSegmentBonus_Marquee', 'MarqueePlanScore']].head()


,AvgSpend_norm,Tier_AB_Fraction,BasketballPropensity_norm,FanSegmentBonus_Marquee,MarqueePlanScore
0,1.000000,0.0,0.872,0.75,0.624400
1,0.590331,1.0,0.485,0.00,0.574099
2,0.544529,0.0,0.896,0.00,0.342559
3,0.137405,0.0,0.467,0.40,0.214621
4,0.071247,0.0,0.582,0.00,0.137774


In [19]:
# Define the bonus mapping for the WeekendPlan score.
bonus_map_weekend = {
    'B': 0.8,
    'E': 1.0,
    'G': 0.5,
    'F': 0.4  # Assuming the second E should be F
}

# Map the FanSegment column to bonus values for the weekend plan.
df['FanSegmentBonus_Weekend'] = df['FanSegment'].map(bonus_map_weekend).fillna(0)


In [27]:
# Calculate WeekendPlanScore using the provided formula:
# WeekendPlan_Score = 0.50*(WeekendFraction)
#                   + 0.20*(1 - DistanceToArena_norm)
#                   + 0.10*(AvgSpend_norm)
#                   + 0.20*(FanSegmentBonus_Weekend)

df['WeekendPlanScore'] = (
    0.60 * df['WeekendFraction'] +
    0.10 * (1 - df['DistanceToArena_norm']) +
    0.10 * df['AvgSpend_norm'] +
    0.20 * df['FanSegmentBonus_Weekend']
)


In [28]:
# Display the relevant columns to verify the new WeekendPlanScore column
df[['WeekendFraction', 'DistanceToArena_norm', 'AvgSpend_norm', 
    'FanSegmentBonus_Weekend', 'WeekendPlanScore']].head()


,WeekendFraction,DistanceToArena_norm,AvgSpend_norm,FanSegmentBonus_Weekend,WeekendPlanScore
0,0.0,0.042254,1.000000,0.4,0.275775
1,0.0,0.165493,0.590331,0.0,0.142484
2,0.0,0.021127,0.544529,0.8,0.312340
3,0.0,0.010563,0.137405,0.0,0.112684
4,0.0,0.014085,0.071247,0.0,0.105716


In [23]:
# Define the bonus mapping for the Promo plan:
# 1 for G, 0.8 for F, 0.4 for E.
bonus_map_promo = {'G': 1.0, 'F': 0.8, 'E': 0.4}

# Map the FanSegment column to bonus values for the Promo plan.
df['FanSegmentBonus_Promo'] = df['FanSegment'].map(bonus_map_promo).fillna(0)


In [24]:
# Calculate PromoPlanScore using the formula:
# PromoPlan_Score = 0.20*(SocialMediaEngagement_norm)
#                + 0.60*(GiveawayFraction)
#                + 0.20*(FanSegmentBonus_Promo)

df['PromoPlanScore'] = (
    0.20 * df['SocialMediaEngagement_norm'] +
    0.60 * df['GiveawayFraction'] +
    0.20 * df['FanSegmentBonus_Promo']
)


In [25]:
# Display the key columns to verify the new PromoPlanScore calculation
df[['SocialMediaEngagement_norm', 'GiveawayFraction', 'FanSegmentBonus_Promo', 'PromoPlanScore']].head()


,SocialMediaEngagement_norm,GiveawayFraction,FanSegmentBonus_Promo,PromoPlanScore
0,0.3,1.0,0.8,0.82
1,0.3,0.0,0.0,0.06
2,0.3,0.0,0.0,0.06
3,1.0,0.0,0.0,0.20
4,0.6,0.0,0.0,0.12


In [29]:
# Save the updated DataFrame with the new ValuePlanScore column to a CSV file
df.to_csv("PlanScores.csv", index=False)
